# Sam Jackson, MAE 494, Homework 5

This code provides answers to the question posed in Homework 5. The packages used in this homework assignment are numpy, matplotlib.pyplot and scipy.optimize.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

Implement an SQP algorithm with line search to solve this problem, starting form $x_0=(1,1)^T$. Incorporate the QP subproblem. Use BFGS approximation for the Hessian of the Lagrangian. Use the merit function and Armijo Line Search to find the step size.

$$\begin{aligned} &\text{min:} && f = x_1^2+(x_2-3)^2 \\ &\text{s.t.} && g_1 = x_2^2-2x_1 \le 0 \\ &&& g_2 = (x_2-1)^2 + 5x_1-15 \le 0 \end{aligned}$$

In [146]:
# Setting the convergence criteria, eps
eps = 1e-3

# Setting the initial guesses for x,mu,W
x0 = [1,1] # Given in problem statement

mu0 = [0,0] # Because no inequality constraints are violated at the start

W = np.identity(np.size(x0)) # Using identity matrix as initial guess

k=0 # Establishing that this is iteration 0

# Establishing the function calls relevant to the code.

f = lambda x:\
    np.power(x[0],2) + np.power(x[1]-3,2)

h = lambda x:\
    np.array([[np.power(x[1],2)-2*x[0]],[np.power(x[1]-1,2)+5*x[0]-15]])

L = lambda x, mu:\
    f(x)+mu[0]*g1(x)+mu[1]*g2(x)

fx = lambda x:\
    np.array([2*x[0],2*(x[1]-3)])

hx = lambda x:\
    np.array([[-2,2*x[1]],[5,2*(x[1]-1)]])

Lx = lambda x, mu:\
    fx(x)+mu[0]*gx(x)[0,:]+mu[1]*gx(x)[1,:]

# Solve QP subproblem
[s,mu]=solveqp(x0,W)

# Line search


# x(k) = x(k-1) + a*s

# BFGS

# k=k+1

# If KKT condition not satisfied, return to QP subproblem


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [145]:
# Functions

def linesearch(x,s,mu_old,w_old):
    t=0.1
    b=0.8
    a=1
    
    D=s
    
    w=np.maximum(np.abs(w_old),0.5*(w_old+np.abs(mu_old)))
    
    count=0
    while count<100:
        phi_a=f(x+a*D)+w*np.abs(np.minimum(0,-g(x+a*D)))
        
        phi0=f(x)+w*abs(np.minimum(0,-g(x)))
        dphi0=fx(x)*D+w*(hx(x)*D)*(g(x)>0)
        psi_a=phi0+t*a*dphi0
        
        if phi_a<psi_a:
            break
        else:
            a=a*b
            count=count+1

def solveqp(x,W):
    #preloop
    A0=hx(x)
    b0=h(x)
    A=[]
    b=[]
    c=fx(x)
    stop=0
    active=[]
    
    #loop
    while stop!=1:
        mu0=np.zeros(np.size(b0,0))
        A=A0[active]
        b=b0[active]
        
        [s,mu]=solveqpactive(x,W,A,b,c)
        
        mu=around(mu*1e12)/1e12
        mu=mu0(active)
        gcheck=A0*s-b0
        gcheck=around(gcheck*1e12)/1e12
        mucheck=0
        
        iadd=[]
        iremove=[]
    
        if mu.size==0:
            mucheck=1
        elif np.min(mu)>0:
            mucheck=1
        else:
            iremove=np.argmin(mu)
    
        if np.amax(gcheck)<=0:
        
            if mucheck==1:
                stop=1
        else:
            iadd=np.argmax(gcheck)
        
        active=setdiff1d(active,active[iremove])
        active=[active,iadd]
    
        active=unique[active]
    
    return [s,mu]


def solveqpactive(x,W,A,b,c):
        M=np.array([np.concatenate([W,A.T],axis=1),np.concatenate([A,np.zeros(A.size)])])
        U=np.array([-c,b])
        sol=np.linalg.solve(M,U)
        s=sol[:np.size(x,0)]
        mu=sol[(np.size(x,0)+1):]
        return [s,mu]